# Install the packages before going futher

In [ ]:
!pip install openai langchain langchain-openai

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.memory import ChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
import datetime
import json

import warnings
warnings.filterwarnings("ignore")

# Load the chatmodel and api key for this notebook

In [ ]:
chat = ChatOpenAI(api_key="sk-proj-jMSHSP0Wtjorx0IwD0atT3BlbkFJ1FLycDvsLR2SDSRcStGr",
    model="gpt-3.5-turbo-1106")

demo_ephemeral_chat_history = ChatMessageHistory()
current_date = datetime.datetime.now().date()

# Prompt and Message history being setup with memory for last 4 messages only

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            f"""You are a helpful assistant designed to output JSON.

                Description:
                You're tasked with creating an application powered by a Language Model (LLM) that processes user queries regarding company performance metrics. Your application should take user input, extract key information such as the company name, performance metric, and time period, and organize it into a structured JSON format.

                Requirements:
                Entity: The name of the company mentioned in the query, with the first letter capitalized.
                Parameter: The performance metric specified in the query.
                Start Date: The beginning date of the requested time period.
                End Date: The end date of the requested time period.
                If the user query does not explicitly state the start date and/or end date, assume the following defaults:

                Start Date: Today's date: {current_date} minus one year.
                End Date: {current_date}.
                Handle variations in user queries, such as different spellings or abbreviations of company names and metric parameters. Consider adding support for additional date formats or relative date ranges (e.g., "last quarter", "previous month").

                The extracted information should be formatted into a JSON structure with the following schema:
                "entity": "<Company Name>",
                "parameter": "<Performance Metric>",
                "startDate": "<Start Date in YYYY-MM-DD format>",
                "endDate": "<End Date in YYYY-MM-DD format>"

                Note: Replace <Company Name> and <Performance Metric> with actual company names and performance metrics extracted from user queries. Ensure the application handles variations in user input for company names, metric parameters, and date formats. Additionally, ensure that the company names always start with a capital letter.

                Use the following query as your input""",
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)

chain = prompt | chat | StrOutputParser()

chain_with_message_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: demo_ephemeral_chat_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [ ]:
def trim_messages(chain_input):
    stored_messages = demo_ephemeral_chat_history.messages
    if len(stored_messages) <= 4:
        return False

    demo_ephemeral_chat_history.clear()

    for message in stored_messages[-4:]:
        demo_ephemeral_chat_history.add_message(message)

    return True


chain_with_trimming = (
    RunnablePassthrough.assign(messages_trimmed=trim_messages)
    | chain_with_message_history
)

# Run this to try out the queries and if you want to get out of the loop enter "bye"

In [ ]:
response = []

while True:
    try:
        user_input = input("Enter input: ")
        if user_input.lower() == "bye":
            break

        response.append(chain_with_trimming.invoke(
            {"input": user_input},
            {"configurable": {"session_id": "unused"}},
        ))

        json_object = json.loads(response[-1])
        json_list = [json_object]
        print(json.dumps(json_list, indent=3))
    except Exception as e:
        print("Try again. Unable to extract necessary information from the query.")

Enter input: Get me flipkart's GMV for the last one year


[
   {
      "entity": "Flipkart",
      "parameter": "GMV",
      "startDate": "2023-05-10",
      "endDate": "2024-05-10"
   }
]
Enter input: compare this with amazon


[
   {
      "entity": "Amazon",
      "parameter": "GMV",
      "startDate": "2023-05-10",
      "endDate": "2024-05-10"
   }
]
Enter input: try wamart too


[
   {
      "entity": "Walmart",
      "parameter": "GMV",
      "startDate": "2023-05-10",
      "endDate": "2024-05-10"
   }
]
Enter input: bye
